# Single Criteria CF

## Preprocessing

In [1]:
import pandas as pd
import numpy as np
from numpy import int64
import statistics
import os
import time

import dill
file_session = 'tmp/singleCF_session.db'
# dill.load_session(file_session)

start_time = time.time()

In [2]:
# import json

ratings_df = pd.read_json("./datasets/top10cat_balanced.json", lines=True)
# ratings_df = pd.DataFrame("./data_fix.json", lines=False)

ratings_df.head(n=10)

ratings_df_filtered = ratings_df.copy(deep=True)
ratings_df_filtered = ratings_df_filtered[['review_id','product_id','product_name','user_id','review_rating','review_packaging','review_rebuy','review_valueformoney']]
ratings_df_filtered['review_rebuy'].replace(['yes','maybe', 'no'],['5','3','1'],inplace=True)
ratings_df_filtered['review_valueformoney'].replace(['expensive','just right','cheap'],['1','3','5'],inplace=True)

# display(ratings_df_filtered)


# =============================================================================
# for key,value in ratings_df_filtered.items():
#     print (key)
#     print(value)
# =============================================================================

    
# print(type(ratings_df_filtered))
user_ids_all = set(ratings_df_filtered.user_id)
product_ids = set(ratings_df_filtered.product_id)
review_ids = set(ratings_df_filtered.review_id)


table = pd.pivot_table(ratings_df_filtered, index='user_id', columns='product_id', values='review_rating')
# cols_to_int = []


count= table.count(axis=1)
countBool = table.count(axis=1) >= 5
countTrue = countBool.sum()

tableFiltered = table[table.count(axis=1) >= 5]
tableFiltered = tableFiltered.fillna(0)
tableFiltered = tableFiltered.astype(int)


path = os.getcwd()
filedir = path+'/outputs/data.csv' # user-row
# pd.to_numeric(tableFiltered)
tableFiltered.to_csv(filedir, index=True, sep=';')

tableFilteredTrans = tableFiltered.T
filedir = path+'/outputs/data_transposed.csv' # product-row
tableFilteredTrans.to_csv(filedir, index=True, sep=';')

print(filedir)

# series1 = tableFiltered.loc[21]
# series = tableFiltered.iloc[21]
# series2 = tableFilteredTrans[21].squeeze()
# series3 = tableFilteredTrans.loc[:,21]

/Users/dhiaaziz/Documents/Kuliah/TA extended/implementasi/outputs/data_transposed.csv


## Kalkulasi Correlation
Step 1 - Calculate Correlation
  desc:
      hitung korelasi antar user menggunakan tableFilteredTrans[0].corr(tableFilteredTrans[1]), looping untuk semua user
      
  output:
  
      correlation_data (pd.DataFrame) - user_id|user_neighbor_id|correlation_coefficient|sorted_rank
  sample:
  
      user_id|user_neighbor_id|correlation_coefficient|rank
      user750|user46657|0.541|1
      user750|user8456|0.537|2
      user750|user76272|0.511|3
      user67|user6680|0.619|1
      user67|user9671|0.597|2
      user67|user25452|0.563|3

In [3]:
correlation_data_collection = []
# correlation_df = pd.DataFrame(columns=['user_id', 'user_neighbor_id', 'correlation_coefficient', 'rank'])
# df_temp = pd.DataFrame(data=[[1,1,1,1]], columns=['user_id', 'user_neighbor_id', 'correlation_coefficient', 'rank'])
# correlation_data = correlation_data.append(df_temp, ignore_index=True)
# correlation_data.append([])

for i in tableFiltered.index:
    
    # for experiment purpose, get only 5 users
#     if (i >= 1000):
#         break
    
#     print(f'Calculate Correlation for User {i}')
    series1 = tableFiltered.loc[i] 
    #iterasi sejumlah user id
    for j in tableFiltered.index:
        if j == i:
            continue
        series2 = tableFilteredTrans.loc[:,j]
        corr = series1.corr(series2, method='pearson')
        # correlation_data.append(corr)
        corr_data = {'user_id': i, 'user_neighbor_id': j, 'correlation_coefficient': corr}
        # correlation_df = correlation_df.append(corr_data, ignore_index=True)
        correlation_data_collection.append(corr_data)
        
correlation_df = pd.DataFrame(correlation_data_collection)


    
correlation_df['rank'] = correlation_df.groupby('user_id')['correlation_coefficient'].rank(ascending=False)

# correlationn = tableFiltered.corr(method='pearson')



In [4]:
correlation_df

,user_id,user_neighbor_id,correlation_coefficient,rank
0,21,69,-0.004609,386.0
1,21,750,0.070953,114.0
2,21,762,0.021847,196.0
3,21,944,-0.000699,257.0
4,21,1218,-0.004089,287.0
...,...,...,...,...
419251,63030,62753,-0.005604,585.0
419252,63030,62757,-0.007205,623.0
419253,63030,62821,-0.003024,318.0
419254,63030,62824,-0.006260,606.0


## Populasi Rating Neighbor

==============================

Step 2 - Populasi Rating Neighbor
  desc:
      koleksi rating yang diberikan oleh 30 tetangga terdekat, dengan cara filter correlation_data kolom rank < 30

  output:
  
      neighbor_rating (pd.DataFrame) - user_id|user_neighbor_id|rank|product_id|neighbor_rating

  sample:
  
      user_id|user_neighbor_id|rank|product_id|neighbor_rating
      user750|user46657|1|product_1|0
      user750|user8456|2|product_1|1
      user750|user76272|3|product_1|0
      user750|user46657|1|product_2|5
      user750|user8456|2|product_2|4
      user750|user76272|3|product_2|0
      user750|user46657|1|product_3|4
      user750|user8456|2|product_3|4
      user750|user76272|3|product_3|4
      
==============================



In [5]:
print('Neighbor\'s Rating')
N = 30#neighbor count
correlation_df_filtered = correlation_df[correlation_df['rank'] <= N]

correlation_df_filtered = correlation_df_filtered.reset_index()
correlation_df_filtered = correlation_df_filtered.drop(columns = ['index'])
correlation_df_filtered['user_id'] = correlation_df_filtered['user_id'].astype(int)
correlation_df_filtered['user_neighbor_id'] = correlation_df_filtered['user_neighbor_id'].astype(int)
correlation_df_filtered['rank'] = correlation_df_filtered['rank'].astype(int)

user_ids = sorted(set(correlation_df_filtered.user_id))

neighbor_rating_collection = []
# neighbor_rating_df = pd.DataFrame(columns=['user_id', 'user_neighbor_id', 'rank', 'product_id', 'neighbor_rating'])
for user_id in user_ids:
    neighbor_per_user_df = correlation_df_filtered[correlation_df_filtered['user_id'] == user_id]
    
    for neighbor_id in neighbor_per_user_df['user_neighbor_id']:
#         print(f'Collecting Rating from User {user_id} and Neighbor {neighbor_id}')
        rankValue = neighbor_per_user_df.loc[(neighbor_per_user_df.user_id == user_id) & (neighbor_per_user_df.user_neighbor_id == neighbor_id)]['rank']
        rankValue = rankValue.item()
        
        for product_id in product_ids:
            # print(f'product {product_id}')
            neighbor_rating = tableFilteredTrans[neighbor_id].loc[product_id] # tableFilteredTrans[column].loc[index]
            neighbor_data = {'user_id': user_id, 'user_neighbor_id': neighbor_id, 'rank': rankValue , 'product_id': product_id, 'neighbor_rating': neighbor_rating}
            neighbor_rating_collection.append(neighbor_data)
            # neighbor_rating_df = neighbor_rating_df.append(neighbor_data, ignore_index=True)
neighbor_rating_df = pd.DataFrame(neighbor_rating_collection)
        
neighbor_rating_df

Neighbor's Rating


,user_id,user_neighbor_id,rank,product_id,neighbor_rating
0,21,3415,4,16385,0
1,21,3415,4,16420,0
2,21,3415,4,8237,0
3,21,3415,4,8239,0
4,21,3415,4,8246,0
...,...,...,...,...,...
36485121,63030,56654,11,8108,0
36485122,63030,56654,11,8120,0
36485123,63030,56654,11,8122,0
36485124,63030,56654,11,8138,0


## Hitung Rekomendasi 

==============================

Step 3 - Hitung Rekomendasi
  desc:
      hitung agregat rating dari semua neighbor

  output:
  
      predicted_rating (pd.DataFrame) - user_id|product_id|predicted_score

  sample:
  
      user_id|product_id|predicted_score
      user750|product_1|0.333
      user750|product_2|3
      user750|product_3|4
      user67|product_1|0.
      user67|product_2|3.
      user67|product_3|4.
      
==============================

In [6]:
predicted_ratings = []
  
for user_id in user_ids: 
#     print(f'Predict items rating for user: {user_id} ')
    temp_user_df = neighbor_rating_df[neighbor_rating_df['user_id'] == user_id]
    for product_id in sorted(product_ids, reverse=True):
        # print(product_id)
        temp_product_df = temp_user_df[temp_user_df['product_id'] == product_id]
        rating_series = temp_product_df['neighbor_rating']
        predicted_rating = rating_series.mean()
        predicted_data = {'user_id': user_id, 'product_id': product_id, 'predicted_score': predicted_rating}
        predicted_ratings.append(predicted_data)        

predicted_rating_df = pd.DataFrame(predicted_ratings)

predicted_rating_df

,user_id,product_id,predicted_score
0,21,45146,0.0
1,21,34186,0.0
2,21,26234,0.0
3,21,20405,0.0
4,21,20098,0.0
...,...,...,...
1216291,63030,89,0.0
1216292,63030,88,0.0
1216293,63030,85,0.0
1216294,63030,73,0.0


## Evaluasi Error

==============================

Step 4 - Evaluasi Error
   desc:
      hitung selisih antara predicted rating dengan actual rating dari user
 
   output:
   
      mean_absolute_error (float) - 0.0

   sample:
   
      0.140212
      
==============================

In [7]:
print("Calculating MAE")
list_mae = []
for user_id in user_ids:
    temp_user_error_df = predicted_rating_df[predicted_rating_df['user_id'] == user_id].sort_values(by=['product_id'])
    temp_actual_rating_series = tableFiltered.loc[user_id].sort_index()

    temp_predicted_rating_series = temp_user_error_df.loc[:,['product_id','predicted_score']].set_index('product_id')
    temp_predicted_rating_series = temp_predicted_rating_series.loc[:, 'predicted_score']

    #pengurangan series / matrix harus memiliki index yang sama
    temp_errors = temp_predicted_rating_series - temp_actual_rating_series
    temp_absolute_errors = temp_errors.abs()
    temp_mae = temp_absolute_errors.mean()
    list_mae.append(temp_mae)
    
mean_absolute_error = statistics.mean(list_mae)



mean_absolute_error

Calculating MAE


0.0556873330667618

In [8]:
print("--- %s seconds ---" % (time.time() - start_time))

--- 1651.240826845169 seconds ---


In [9]:
dill.dump_session(file_session)

KeyboardInterrupt: 

In [ ]:
# mean_absolute_error

In [ ]:
# dill.load_session(file_session)